## McCulloch-Pitts Network

The first neural network, the McCulloch-Pitts network, was developed in the 1940s. It takes a set of inputs, multiply these inputs by weights, and then outputs either a 0 or 1. If the sum of the products exceed the threshold, then an  output of 1 is returned. Otherwise, the output value is 0. In the case of 2 inputs, this means:

$$
output = \left\{ \begin{array}{rl}
 1 &\mbox{ if $x_{1}w_{1} + x_{2}w_{2} \geq threshold$} \\
 0 &\mbox{ if $x_{1}w_{1} + x_{2}w_{2} < threshold$}
       \end{array} \right.
$$

Note that we can move the threshold to the left side:

$x_{1}w_{1} + x_{2}w_{2} - threshold \geq 0\\$

And we can define a new term, the bias, as $b=-threshold$. For this entire tutorial, we will use the notation $w_{0}$ to denote the bias instead of $b$. We will also add another $x_{0}$ unit, the bias unit, which always equals 1, to go with the weight. So now the equation becomes: 

$x_{0}w_{0} + x_{1}w_{1} + x_{2}w_{2} \geq 0$

With this, we have to define three weights - $w_{0}$, $w_{1}$, $w_{2}$ - and we don't have to worry about the threshold.

This type of network cannot learn--that is, the values of the weights must be determined beforehand and are fixed. Even so, they can be used to compute simple logical operators.

Let us first try to make some logical operators using this simple, two-layer neural network.

In [2]:
def McCulloch(weights):
    '''Given a list of 3 weights [bias, w1, w2], return a
    truth table dictionary'''
    
    # Initialization
    truth_table = {(0,0):0, (0,1):0, (1,0):0, (1,1):0}
    
    for x1, x2 in truth_table:
        # Sum of products of weight and input
        output = weights[0] + x1*weights[1] + x2*weights[2]
        
        # Output exceeds threshold
        if output >= 0:
            truth_table[(x1, x2)] = 1
    
    return truth_table

def displayTable(truth_table):
    '''Given a truth table dictionary, display it nicely'''
    
    print()
    print(' --------------------------')
    print('|   x1   |   x2   | Output |')
    print('|--------------------------|')
    
    for x1, x2 in truth_table:
        print('|    {0}   |    {1}   |    {2}   |'.format(x1,x2,truth_table[(x1,x2)]))
        
    print(' --------------------------')
    
def determineLogic(truth_table):
    '''Given a truth table dictionary, determine the
    logical operator it represents'''

    if truth_table[(0,0)] and truth_table[(1,1)]:
        return 'invalid'
    
    elif truth_table[(0,0)]:
        if truth_table[(0,1)] and truth_table[(1,0)]:
            return 'NAND'
        elif truth_table[(0,1)]:
            return 'NOT x1'
        elif truth_table[(1,0)]:
            return 'NOT x2'
        else:
            return 'NOR'
    
    elif truth_table[(1,1)]:
        if truth_table[(0,1)] and truth_table[(1,0)]:
            return 'OR'
        elif not (truth_table[(0,1)] or truth_table[(1,0)]):
            return 'AND'
        else:
            return 'invalid'
    
    else:
        return 'invalid'


Try varying the values of the weights below!

(The weight values has been set to give the AND operator.)

In [3]:
#################################
# Vary the values of the weights!
w0 = 0
w1 = 0
w2 = -2
#################################

truth_table = McCulloch([w0,w1,w2])
displayTable(truth_table)
print('Logical operator:', determineLogic(truth_table))


 --------------------------
|   x1   |   x2   | Output |
|--------------------------|
|    0   |    1   |    0   |
|    1   |    0   |    1   |
|    0   |    0   |    1   |
|    1   |    1   |    0   |
 --------------------------
Logical operator: NOT x2


Note that you would be able to create the AND, NAND, OR, NOR, and NOT gates.

To create the XOR and XNOR gate, you would need another hidden layer.

## Perceptrons

In the late 1950's, perceptrons were developed. For perceptrons, the threshold value can vary rather than being fixed, but the general idea is the same. The output value will be 1 if the sum of products of inputs and weights are greater or equal to the threshold, otherwise the output value is 0.

The biggest difference between the perceptron and the McCulloch-Pitts network is that it is a learning algorithm. This means that you won't have to specify the weights anymore - as long as you can give the correct outputs to input, with enough iterations and training examples, the network can learn the weights by itself!

The learning algorithm is as follows:

1. If Output is correct, then no changes are made to the threshold or weights.
2. If Output = 1 but should be 0, then increase threshold and decrease the weights.
3. If Output = 0 but should be 1, then decrease threshold and increase the weights.

The weights are only changed if the corresponding input is 1. That is, if $x_{1}=0$ and $x_{2}=1$, only $w_{2}$ will be updated.

The learning algorithm (designed for computing logical operators) is implemented below. Note that the returned weights are in the format of $[-threshold, w_{1}, w_{2}]$. As explained earlier, $bias = -threshold$.

In [4]:
from random import choice

def perceptrons(truth_table, i):
    '''Given a truth table dictionary, return the weights and threshold'''
    
    # Initialize weights and threshold
    weights = [0,0]
    t = 0
    
    for iterations in range(i):
        
        # Randomly choose the inputs and correct output
        x1, x2 = choice(list(truth_table.keys()))
        correct_output = truth_table[(x1,x2)]
        
        # Determine whether output exceeds threshold
        if (weights[0]*x1 + weights[1]*x2) >= t:
            output = 1
        else:
            output = 0
        
        # Learning algorithm: update the weights and threshold
        # Only update weights if x1 or x2 is 1
        weights[0] += (correct_output-output)*x1
        weights[1] += (correct_output-output)*x2
        t -= (correct_output-output)
    
    # Corresponds to [w0, w1, w2] in McCulloch-Pitts
    return [-t] + weights

def LogicToTruthTable(logic):
    '''Given the logical operator, return the truth table
    corresponding to that operator'''
    
    logic = logic.upper()
    
    if logic == 'AND':
        truth_table = {(0,0):0, (0,1):0, (1,0):0, (1,1):1}
        
    elif logic == 'OR':
        truth_table = {(0,0):0, (0,1):1, (1,0):1, (1,1):1}
        
    elif logic == 'NOR':
        truth_table = {(0,0):1, (0,1):0, (1,0):0, (1,1):0}
    
    elif logic == 'NAND':
        truth_table = {(0,0):1, (0,1):1, (1,0):1, (1,1):0}
    
    elif logic == 'NOT X1':
        truth_table = {(0,0):1, (0,1):1, (1,0):0, (1,1):0}
    
    elif logic == 'NOT X2':
        truth_table = {(0,0):1, (0,1):0, (1,0):1, (1,1):0}
    
    else:
        return 'invalid input'
    
    return truth_table

Now you can try to experiment by changing the logical operators below (AND, NAND, OR, NOR, NOT x1, and NOT x2), and see what values of weights are returned!

The program also double-checks these weight values into the McCulloch-Pitts network.

In [5]:
###############################################
# Try varying the logical operators!
logical_operator = 'AND'

###############################################

print('Given logical operator: ', logical_operator)
truth_table = LogicToTruthTable(logical_operator)
weights = perceptrons(truth_table, 1000)
print('Weights calculated by perceptron:', weights)
print()

# Double checking
print('Entering the calculated weights into the McCulloch Pitts network...')
truth_check = McCulloch(weights)
displayTable(truth_check)
print('Logical operator:', determineLogic(truth_check))

print()
if determineLogic(truth_check).upper() == logical_operator.upper():
    print('Perceptrons performed successfully.')
    
else:
    print('Perceptrons failed to perform successfully.')


Given logical operator:  AND
Weights calculated by perceptron: [-3, 1, 2]

Entering the calculated weights into the McCulloch Pitts network...

 --------------------------
|   x1   |   x2   | Output |
|--------------------------|
|    0   |    1   |    0   |
|    1   |    0   |    0   |
|    0   |    0   |    0   |
|    1   |    1   |    1   |
 --------------------------
Logical operator: AND

Perceptrons performed successfully.


Now, let us move on to the next section: multi-layer neural networks!

## References & Further Readings

- [A Basic Introduction to Neural Networks](http://www.webpages.ttu.edu/dleverin/neural_network/neural_networks.html)
- [Coursera: Machine Learning](https://www.coursera.org/learn/machine-learning)
- [Neural Networks and Deep Learning: Chapter 1](http://neuralnetworksanddeeplearning.com/chap1.html)
- [Single-layer Neural Networks (Perceptrons)](http://computing.dcu.ie/~humphrys/Notes/Neural/single.neural.html)
- [Logic Gates](http://whatis.techtarget.com/definition/logic-gate-AND-OR-XOR-NOT-NAND-NOR-and-XNOR)